# Load general packages and definitions

In [ ]:
## general
import numpy as np
import datetime
from sklearn.externals import joblib
import copy
import cf_units
import xarray as xr
import os
import sys
from tqdm import tqdm_notebook as tqdm
import datetime as dt


## statistics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.stats import multivariate_normal # to compute likelihood
from scipy.optimize import curve_fit, fmin, fminbound, minimize, rosen_der
from sklearn.preprocessing import StandardScaler

# statistics which aren't all that nice in python
import rpy2.robjects as robjects

## plotting
import matplotlib.pyplot as plt
import numpy.ma as ma
import cartopy.crs as ccrs

##for parallelisation
from sklearn.externals.joblib import Parallel, delayed

In [ ]:
##import functions for fitting
import os
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
from symfit import parameters, variables, Fit
from symfit import pi,sqrt,log,exp,sinh
from symfit import sin, cos


##define function to do fitting on
def season_mimic(x, mon, n=2):
    """
    Returns a symbolic fourier series of order `n`.

    :param n: Order of the fourier series.
    :param x: Independent variable
    :param f: Frequency of the fourier series
    """
    # Make the parameter objects for all the terms
    a = parameters(','.join(['a{}'.format(i) for i in range(1, n + 1)]))
    b = parameters(','.join(['b{}'.format(i) for i in range(1, n + 1)]))
    c = parameters(','.join(['c{}'.format(i) for i in range(1, n + 1)]))
    d = parameters(','.join(['d{}'.format(i) for i in range(1, n + 1)]))
   
    # Construct the series
    series =    sum((ai*x + bi)*sin(pi*i*(mon%12+1)/6)+(ci*x+di)*cos(pi*i*(mon%12+1)/6)for i,(ai, bi, ci, di) in enumerate(zip(a, b,c,d)))
    return series



In [ ]:
np.pi

In [ ]:
def fit_to_bic(x_train,mon_train,y_all_mon,max_period=10):
    
    n=len(x_train)
    bic=np.zeros(max_period)
    for i in range (1,max_period+1):
        
        num_params=2+(4*i)
        x, mon, z = variables('x, mon, z')
        model_dict = {z: season_mimic(x, mon, n=i)}
        
        if i==1:
            fit=Fit(model_dict, x=x_train, z=y_all_mon)
            mse=mean_squared_error(y_all_mon,fit.model(x=x_train, **fit.execute().params).z)
        else:
            fit=Fit(model_dict, x=x_train, mon=mon_train,z=y_all_mon)
            mse=mean_squared_error(y_all_mon,fit.model(x=x_train, mon=mon_train,**fit.execute().params).z)
        
        bic[i-1] = n * log(mse) + num_params * log(n)
        
    order_chosen=np.where(bic==min(bic))[0][0]+1
    
    
    model_dict_chosen = {z: season_mimic(x, mon, n=order_chosen)}
    fit=Fit(model_dict_chosen, x=x_train, mon=mon_train, z=y_all_mon)
    
    return fit.execute()

In [ ]:
def fit_fmin(self, X, y):
    """Estimate the optimal parameter lambda for each feature.
    The optimal lambda parameter for minimizing skewness is estimated on
    each feature independently using maximum likelihood.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        The data used to estimate the optimal transformation parameters.
    y : temp values to calculate lmbda as lmbda = a*y + b
    Returns
    -------
    self : object
    """
    
    X = X.copy()  # force copy so that fit does not change X inplace


    self.coeffs_ = np.array(Parallel(n_jobs=12,verbose=0)(delayed(yeo_johnson_optimize_fmin)(self,X[:,i_grid],y[:,i_grid]) for i_grid in tqdm(np.arange(idx_l.sum()))))
    #print(self.coeffs_.shape)
    
    self.mins_ = np.amin(X, axis=0)
    self.maxs_ = np.amax(X, axis=0)
    
    #print(self.coeffs_)
    
    if self.standardize:
        self._scaler = StandardScaler(copy=True)
        self._scaler.fit(X)
        
    return self
    
def transform_fmin(self, X, y):
        """Apply the power transform to each feature using the fitted lambdas.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The data to be transformed using a power transformation.
        Returns
        -------
        X_trans : array-like, shape (n_samples, n_features)
            The transformed data.
        """

        lambdas=get_yeo_johnson_lambdas(self.coeffs_,y)
        
        X_trans=np.zeros_like(X)
        for i, lmbda in enumerate(lambdas.T):
            for j,j_lmbda in enumerate(lmbda):
                with np.errstate(invalid='ignore'):  # hide NaN warnings
                    X_trans[j, i] = self._yeo_johnson_transform(X[j, i], j_lmbda)

        if self.standardize:
            X_trans = self._scaler.transform(X_trans)

        return X_trans

def inverse_transform_fmin(self, X, y):
        """Apply the inverse power transformation using the fitted lambdas.
        The inverse of the Yeo-Johnson transformation is given by::
            if X >= 0 and lambda_ == 0:
                X = exp(X_trans) - 1
            elif X >= 0 and lambda_ != 0:
                X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1
            elif X < 0 and lambda_ != 2:
                X = 1 - (-(2 - lambda_) * X_trans + 1) ** (1 / (2 - lambda_))
            elif X < 0 and lambda_ == 2:
                X = 1 - exp(-X_trans)
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The transformed data.
        Returns
        -------
        X : array-like, shape (n_samples, n_features)
            The original data
        """

        if self.standardize:
            X = self._scaler.inverse_transform(X) 
        
        X_inv = np.zeros_like(X)
        
        lambdas=get_yeo_johnson_lambdas(self.coeffs_,y)
        for i, lmbda in enumerate(lambdas.T):
            for j,j_lmbda in enumerate(lmbda):
                with np.errstate(invalid='ignore'):  # hide NaN warnings
                    X_inv[j, i] = self._yeo_johnson_inverse_transform(X[j, i], j_lmbda)
            X_inv[:,i]=np.where(X_inv[:,i]<self.mins_[i],self.mins_[i],X_inv[:,i])
            X_inv[:,i]=np.where(X_inv[:,i]>self.maxs_[i],self.maxs_[i],X_inv[:,i])

        return X_inv

def yeo_johnson_optimize_fmin(self, x, y):
    """Find and return optimal lambda parameter of the Yeo-Johnson
    transform by MLE, for observed data x.
    Like for Box-Cox, MLE is done via the brent optimizer.
    """

    def _neg_log_likelihood(coeff):
        """Return the negative log likelihood of the observed data x as a
        function of lambda."""
        lambdas=2/(1+coeff[0]*np.exp(y*coeff[1]))
        
        x_trans =np.zeros_like(x)
        
        #print(lambdas.shape)
        for i, lmbda in enumerate(lambdas):
            x_trans[i] = self._yeo_johnson_transform(x[i], lmbda)
        
        n_samples = x.shape[0]

        loglike = -n_samples / 2 * np.log(x_trans.var())
        loglike += ((lambdas - 1) * np.sign(x) * np.log1p(np.abs(x))).sum()

        return -loglike

    # the computation of lambda is influenced by NaNs so we need to
    # get rid of them
    x = x[~np.isnan(x)]
    # choosing bracket -2, 2 like for boxcox
    bounds=np.c_[[0,0], [1,0.1]]
    
    return minimize(_neg_log_likelihood, np.array([0.01,0.001]), bounds=bounds, method='SLSQP',jac=rosen_der 
               ).x

def get_yeo_johnson_lambdas(coeffs,y):

    lambdas=np.zeros_like(y)
    i=0
    for a,b in zip(coeffs,y.T):
        
        lambdas[:,i]=2/(1+a[0]*np.exp(b*a[1]))
        i+=1
        
    lambdas=np.where(lambdas<0,0,lambdas)
    lambdas=np.where(lambdas>2,2,lambdas)
    
    return lambdas
        
        

In [ ]:
from sklearn.preprocessing import PowerTransformer

def power_fit(residue,y,fmin=True):
    
    if fmin:
        power_trans = fit_fmin(PowerTransformer(method='yeo-johnson'),residue.reshape(-1, idx_l.sum()),y)
    else:
        power_trans = PowerTransformer(method='yeo-johnson').fit(residue.reshape(-1, idx_l.sum()))
    
    return power_trans
    
def power_transform(mod, residue,y,fmin=True):
    
    if fmin:
        residue_trans = transform_fmin(mod,residue.reshape(-1, idx_l.sum()),y).reshape(-1,idx_l.sum())
    else:
        residue_trans = mod.transform(residue.reshape(-1, idx_l.sum())).reshape(-1,idx_l.sum())
            
    return residue_trans  

def power_inv_transform(mod, residue,y,fmin=True):
    
    if fmin:
        residue_inv_trans = inverse_transform_fmin(mod,residue.reshape(-1, idx_l.sum()),y).reshape(-1,idx_l.sum())
    else:
        residue_inv_trans = mod.inverse_transform(residue.reshape(-1, idx_l.sum())).reshape(-1,idx_l.sum())
            
    return residue_inv_trans  

def compute_llh_cv(res_tr,res_cv,phi):
    """ Compute sum of log likelihood of a set of residuals based on a covariance matrix derived from a different set (of timeslots) of residuals
    
    Keyword arguments:
        - res_tr: the residual of the training run lacking a specific fold after removing the local mean response (nr ts x nr gp). Nans must be removed before
        - res_cv: the residual of a fold which was removed from the training run
        - phi: matrix to localize the covariance matrix based on a specific localisation radius and distance information (phi = output of fct gaspari_cohen(geo_dist/L))
    
    Output:
        - llh_innov_cv: sum of the log likelihood over the cross validation time slots
    
    """

    ecov_res_tr = np.cov(res_tr,rowvar=False)
    cov_res_tr=phi*ecov_res_tr
    
    mean_0 = np.zeros(phi.shape[0]) # we want the mean of the res to be 0

    llh_innov_cv=np.sum(multivariate_normal.logpdf(res_cv,mean=mean_0, cov=cov_res_tr,allow_singular=True))

    return llh_innov_cv   

def leave_one_out(L_set,nr_folds,residue_trans,idx_fo_tot,phi):
    
    def folds_calc(idx_fo,residue_trans,phi,L):
    
        res_tot_est = residue_trans[~idx_fo] 
        res_tot_fo=residue_trans[idx_fo]

        llh_cv=compute_llh_cv(res_tot_est,res_tot_fo,phi[L])
        
        return llh_cv
    
    idx_L=0
    L = L_set[idx_L]
    
    df_llh_cv={}
    df_llh_cv['llh_max']=-10000
    df_llh_cv['all']={}
    df_llh_cv['sum']={}
    df_llh_cv['L_sel']=L_set[idx_L]
    
    while (L-df_llh_cv['L_sel']<=250) and (df_llh_cv['L_sel']<L_set[-1]): # based on experience I know that once stop selecting larger 
            #loc radii, will not start again -> no point in looping through everything, better to stop once max is 
            #reached (to avoid singular matrices)
        L = L_set[idx_L]
        print('start with L ',L)
        df_llh_cv['all'][L]={}
        df_llh_cv['sum'][L]=0
        for i_fold_par in np.arange(len(idx_fo_tot.keys())):
            df_llh_cv['all'][L][i_fold_par]=folds_calc(idx_fo_tot[i_fold_par],residue_trans,phi,L)
            df_llh_cv['sum'][L] += df_llh_cv['all'][L][i_fold_par]
            
       
        #df_llh_cv['all'][L]=Parallel(n_jobs=10,verbose=10)(delayed(folds_calc)(idx_fo_tot[i],residue_trans,phi,L)for i in np.arange(len(idx_fo_tot.keys())))
        
        #print('rest tot fo shape ',res_tot_fo.shape,'res_tot_est shape ',res_tot_est.shape)
        if df_llh_cv['sum'][L]>df_llh_cv['llh_max']:
            df_llh_cv['L_sel']=L
            df_llh_cv['llh_max']=df_llh_cv['sum'][L]
            print('currently selected L=',df_llh_cv['L_sel'])

        idx_L+=1  
    return df_llh_cv

def lin_func(x, a, b):
    return a * x + b

In [ ]:
def compute_llh_cv(res_tr,res_cv,phi):
    """ Compute sum of log likelihood of a set of residuals based on a covariance matrix derived from a different set (of timeslots) of residuals
    
    Keyword arguments:
        - res_tr: the residual of the training run lacking a specific fold after removing the local mean response (nr ts x nr gp). Nans must be removed before
        - res_cv: the residual of a fold which was removed from the training run
        - phi: matrix to localize the covariance matrix based on a specific localisation radius and distance information (phi = output of fct gaspari_cohen(geo_dist/L))
    
    Output:
        - llh_innov_cv: sum of the log likelihood over the cross validation time slots
    
    """

    ecov_res_tr = np.cov(res_tr,rowvar=False)
    cov_res_tr=phi*ecov_res_tr
    
    mean_0 = np.zeros(phi.shape[0]) # we want the mean of the res to be 0

    llh_innov_cv=np.sum(multivariate_normal.logpdf(res_cv,mean=mean_0, cov=cov_res_tr))

    return llh_innov_cv   



def gaspari_cohn(r):
    """ Localize covariance matrix.

        Keyword argument:
        - r = distance / localization radius

        Remarks    	
        - based on Gaspari-Cohn 1999, QJR  (as taken from Carrassi et al 2018, arxiv)
    
    """
    r = np.abs(r)

    
    if r>=0 and r<1:
            y = 1 - 5/3*r**2 + 5/8*r**3 + 1/2*r**4 - 1/4*r**5
    if r>=1 and r<2:
            y = 4 - 5*r + 5/3*r**2 + 5/8*r**3 - 1/2*r**4 + 1/12*r**5 - 2/(3*r)
    if r>=2:
            y = 0
        
        
    return y


# Loading/Fitting Harmonic model to ESMs

In [ ]:
import mplotutils as mpu

frac_l = xr.open_mfdataset('/net/so4/landclim/snath/data/interim_invariant_lsmask_regrid.nc', combine='by_coords',decode_times=False)
#land-sea mask of ERA-interim bilinearily interpolated 
frac_l_raw = np.squeeze(copy.deepcopy(frac_l.lsm.values))
frac_l = frac_l.where(frac_l.lat>-60,0) # remove Antarctica from frac_l field (ie set frac l to 0)
idx_l=np.squeeze(frac_l.lsm.values)>0.0 # idex land #-> everything >0 I consider land

lon_pc, lat_pc = mpu.infer_interval_breaks(frac_l.lon, frac_l.lat)



In [ ]:

##Start training for monthly downscaling
models=#specify list of model names
nr_ts=#no. of years

y_all_mon={}
y_all={}
run_nrs={}

seasonal_model_exec={}

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

for model in tqdm(models): # model is just the training model, can also be observations e.g. model is "BEST"
        
        print("calculating seasonal trends for" ,model)
        
        # prepare the inputs as array
        y_all[model] = #path to yearly temperature data 
        y_all_mon[model] = #path to monthly temperature data 
        run_nrs[model]=y_all[model].reshape(-1,nr_ts,idx_l.sum()).shape[0]
        ##Get directory to store outputs
        dir_out_data_mod = #path to storage
        
        if not os.path.exists(dir_out_data_mod):
            os.makedirs(dir_out_data_mod)
            print('created dir:',dir_out_data_mod)
    
        
        ##prepare training data
        months=np.arange(1,13)
        mon_train=np.tile(months,y_all[model].shape[0])

        seasonal_model_exec[model]= Parallel(n_jobs=10)(delayed(fit_to_bic)(np.repeat(y_all[model][:,i],12),
                                                                             mon_train, y_all_mon[model][:,i], 12) 
                                                        for i in np.arange(idx_l.sum()))
            
        joblib.dump(seasonal_model_exec[model],dir_out_data_mod+'seasonal_trend.pkl')
            

In [ ]:
##Extract test and train results
sys.path.insert(1,'/home/snath/polybox/LAMACLIMA/')
from tools.loading import load_data_single_mod
import importlib
importlib.reload(sys.modules['tools.loading'])
from tools.loading import load_data_single_mod

train_results_all={}

y_all_mon={}
y_all={}

seasonal_mod={}

order_chosen={}
for model in models:
    
    #Get directory
    dir_in_data_mod = #path to storage
    print("Getting seasonal trends for" ,model)
    
    seasonal_mod[model]=joblib.load(dir_in_data_mod+'seasonal_trend.pkl')
    ##get training data
    
    # prepare the inputs as array
    y_all[model] = #path to yearly temperature data 
    y_all_mon[model] = #path to monthly temperature data 

    if os.path.exists(os.path.join(dir_in_data_mod,'seasonal_training_results.pkl')):
        train_results_all[model]=joblib.load(dir_in_data_mod+'seasonal_training_results.pkl')

    else:

        ##prepare array to store train results
        train_results_all[model]=np.zeros_like(y_all_mon[model] ) 
        months=np.arange(1,13)
        mon_train=np.tile(months,y_all[model].shape[0] )
        for i in np.arange(idx_l.sum()):
            x_train=np.repeat(y_all[model][:,i],12)
            x, mon, z = variables('x, mon, z')
            train_results_all[model][:,i]=seasonal_mod[model][i].model(x=x_train, mon=np.tile(months,nr_ts*len(run_nrs[model])),**seasonal_mod[model][i].params).z
    
    
        ##store training results
        joblib.dump(train_results_all[model],dir_in_data_mod+'seasonal_training_results.pkl')  

    order_chosen[model]=np.zeros([idx_l.sum()]) 

    ##plot order of harmonics chosen
    
    for i in np.arange(idx_l.sum()):
        
        fit_result=seasonal_mod[model][i]
        order_chosen[model][i]=int((len(fit_result.params)+2)/4)-1
        
            
            
    fig=plt.figure(figsize=(10,20))
    plt.rcParams.update({'font.size': 10})
    plt.rcParams.update({'mathtext.default':'regular'}) 
    plt.rcParams.update({'mathtext.default':'it'}) 

    ax=fig.add_subplot(1,1,1,projection=ccrs.Robinson(central_longitude=0))

    y_ma = np.zeros(idx_l.shape)
    y_ma = ma.masked_array(y_ma, mask=idx_l==False)
    y_ma[idx_l]=order_chosen[model]-1

    ax.coastlines()
    mesh_1=ax.pcolormesh(lon_pc, lat_pc, y_ma,  cmap=plt.cm.get_cmap('cubehelix_r',7),vmin=0,vmax=7,transform=ccrs.PlateCarree(),rasterized=True)
    ax.set_title('Fourier Series %s'%model,y=1.02,fontsize=12)

    cbar=plt.colorbar(mesh_1,ax=[ax],orientation='horizontal',ticks=(np.arange(8)+0.5),shrink=0.8,pad=0.02,aspect=25)
    cbar.set_label('order')  
    cbar.ax.set_xticklabels(np.arange(8))
        
        

# Creating local variability module

In [ ]:
## Transform variables first

power_trans={}
train_residue_trans={}
train_residue_all={}
test_residue_all={}

for model in models:
    
    print('Training power transformer for,', model)
    train_residue_all[model]=y_all_mon[model]-train_results_all[model]
    train_residue_all[model]=train_residue_all[model].reshape(-1,12,idx_l.sum())
    
    power_trans[model]=[]
    for i_mon in range(12):
        power_trans[model].append(power_fit(train_residue_all[model][:,i_mon,:],y_all[model]))

    train_residue_trans[model]=Parallel(n_jobs=12,verbose=10)(delayed(power_transform)(power_trans[model][i_mon],train_residue_all[model][:,i_mon,:],y_all[model]) for i_mon in range(12))
    
    temp_residue_trans=np.zeros_like(train_residue_all[model])
    for i_mon in range(12):
        temp_residue_trans[:,i_mon,:]=train_residue_trans[model][i_mon]
    train_residue_trans[model]=temp_residue_trans
    
    dir_out_data_mod = #path to storage

    if not os.path.exists(dir_out_data_mod):
        os.makedirs(dir_out_data_mod)
        print('created dir:',dir_out_data_mod)
    joblib.dump(power_trans[model],dir_out_data_mod+'yeo_johnson_pt_fmin_log.pkl')
    

In [ ]:
##Fit Month-based AR(1) process


train_residue_all={}

coeff_0={}
coeff_1={}

for model in models:
    
    print('start with AR(1) model for',model)
        
    coeff_0[model]=np.zeros([12,idx_l.sum()])
    coeff_1[model]=np.zeros([12,idx_l.sum()])
    
    for i_grid in tqdm(np.arange(idx_l.sum())):
        
        for i_mon in range(12):
            
            if i_mon==0:
                
                ar1_month = curve_fit(lin_func, train_residue_trans[model][:-1,-1:,i_grid].reshape(-1),train_residue_all[model][1:,i_mon,i_grid].reshape(-1), bounds=([-1,-np.inf], [1, np.inf]))[0]##bounded gamma_1 term [-1,1] because otherwise sqrt(1-gamma_1**2) is NAN for calculation of analytic covariance matrix
                
                coeff_0[model][i_mon,i_grid]=ar1_month[1]
                coeff_1[model][i_mon,i_grid]=ar1_month[0]
            
            else:
                
                ar1_month=curve_fit(lin_func, train_residue_trans[model][:,i_mon-1,i_grid].reshape(-1),train_residue_all[model][:,i_mon,i_grid].reshape(-1), bounds=([-1,-np.inf], [1, np.inf]))[0]
                coeff_0[model][i_mon,i_grid]=ar1_month[1]
                coeff_1[model][i_mon,i_grid]=ar1_month[0]
                            
        dir_out_data_mod = #path to storage

        joblib.dump(np.stack((coeff_0[model],coeff_1[model]),axis=0),dir_out_data_mod+'AR(1)_coeffs.pkl')
            
        

In [ ]:
dir_in_geo_dist = '/net/so4/landclim/snath/data/'
geo_dist=np.load(dir_in_geo_dist + 'geo_dist.npy')

L_set = [1500,1750,2000,2250,2500,2750,3000,3250,3500,3750,4000,4250,4500,4750,5000,5250,5500,6000,6250,6500,7000,7500,8000,8500]

phi = {}
for L in L_set:
    phi[L] = np.zeros(geo_dist.shape)

    for i in np.arange(geo_dist.shape[0]):
        for j in np.arange(geo_dist.shape[1]):
            phi[L][i,j]=gaspari_cohn(geo_dist[i,j]/L)
        if i % 1000 == 0:
                print('done with L:',L,'i:', i)
                

In [ ]:
df_llh_cv_all={}
L_set = [1500,1750,2000,2250,2500,2750,3000,3250,3500,3750,4000,4250,4500,4750,5000,5250,5500] 

coeff_0={}
coeff_1={}

for model in models:
        dir_in_data_mod = #path to storage
        
        coeffs_temp = joblib.load(dir_in_data_mod+'AR(1)_coeffs.pkl')
        coeff_0[model] = coeffs_temp[0,:,:]
        coeff_1[model] = coeffs_temp[1,:,:]
        power_trans[model]=joblib.load(dir_in_data_mod+'yeo_johnson_pt_fmin_log.pkl')
        
        AR_process=np.zeros([train_residue_trans[model].reshape(-1,idx_l.sum()).shape[0]+120,
                         idx_l.sum()]).reshape(-1,12,idx_l.sum())
    
        for t in np.arange(1,AR_process.shape[0]):
            for i_mon in range(12):

                if i_mon==0:
                    AR_process[t,i_mon,:]=coeff_0[model][i_mon,:]+coeff_1[model][i_mon,:]*AR_process[t-1,11,:]
                else:
                     AR_process[t,i_mon,:]=coeff_0[model][i_mon,:]+coeff_1[model][i_mon,:]*AR_process[t,i_mon-1,:]

        AR_process= AR_process[10:,:,:]

        
        train_residue_all_spat[model]=train_residue_trans[model].reshape(-1,12,idx_l.sum())-AR_process
    
        

        # hardcoded for very slow leav-1-out cross val at moment to ensure to get most out of the data
        nr_ts=len(time[model])
        nr_folds = nr_ts*len(run_nrs[model])
        print('number folds', nr_folds)
        fold_out_list = np.arange(nr_folds)
        idx_fo_tot={}
        j=0
        for i in fold_out_list:      
            idx_fo = np.zeros(nr_folds,dtype=bool)
            idx_fo[j:j+1]=True
            idx_fo_tot[i]=idx_fo    
            j+=1    

        # carry out cross-validation to determine the localisation radius L
        print('start with localisation radius for',model)

        df_llh_cv_all[model]={}
        df_llh_cv_all[model]=Parallel(n_jobs=12,verbose=10)(delayed(leave_one_out)(L_set,nr_folds,train_residue_all_spat[model][:,i_mon,:],idx_fo_tot,phi) for i_mon in range(12))
        
        dir_out_data_mod = #path to storage

        joblib.dump(df_llh_cv_all[model],dir_out_data_mod+'llh_cv_all.pkl')


# Create Emulations

In [ ]:
emu_res={}
nr_emus=500
buffer=10
nr_ts=#insert no. of years in dataset
dir_in_data_mod = #<in_file_path>
dir_out_data_mod = #<out_file_path>

##load calibration parameters for local variability module
df_llh_cv_all= joblib.load(dir_in_data_mod+'tas_loc_var/llh_cv_all.pkl')
coeffs_temp = joblib.load(dir_in_data_mod+'tas_loc_var/AR(1)_coeffs.pkl')
coeff_0 = coeffs_temp[0,:,:]
coeff_1 = coeffs_temp[1,:,:]
power_trans=joblib.load(dir_in_data_mod+'tas_loc_var/yeo_johnson_pt_fmin_log.pkl')

train_residue_all=(y_all_mon-train_results_all)
train_residue_all=train_residue_all.reshape(-1,12,idx_l.sum())

##transform residuals
train_residue_trans=Parallel(n_jobs=10,verbose=10)(delayed(power_transform)(power_trans[i_mon],train_residue_all[:,i_mon,:],y_all) for i_mon in range(12))
temp_residue_trans=np.zeros_like(train_residue_all)
for i_mon in range(12):
    temp_residue_trans[:,i_mon,:]=train_residue_trans[i_mon]
train_residue_trans=temp_residue_trans


print('calculating covariance matrices')
innov_emu={}
cov_innov={}
for i_mon in tqdm(range(12)):
    L_sel=df_llh_cv_all[i_mon]['L_sel']
    ecov_res=np.cov(train_residue_trans[:,i_mon,:],rowvar=False)
    cov_res=phi[L_sel]*ecov_res
    cov_innov[i_mon]=np.zeros_like(cov_res)
    
    for i in np.arange(idx_l.sum()):
        for j in np.arange(idx_l.sum()):
               cov_innov[i_mon][i,j]=np.sqrt(1-coeff_1[i_mon,i]**2)*np.sqrt(1-coeff_1[i_mon,j]**2)*cov_res[i,j]
    
    cov_innov[i_mon][np.isnan(cov_innov[i_mon])]=0    

print('innovations drawn')
emu_res[model]={}
innov_emu={}
for i_mon in tqdm(range(12)):
    innov_emu[i_mon] = np.random.multivariate_normal(np.zeros(idx_l.sum()),cov_innov[i_mon],size=[nr_emus,nr_ts+buffer],check_valid='warn')

start = dt.datetime.now()

for k in np.arange(nr_emus):
    emu_res[model][k]=np.zeros([nr_ts+buffer,12,idx_l.sum()])
    for t in np.arange(1,emu_res[model][k].shape[0]):
        for i_mon in range(12):

            if i_mon==0:
                emu_res[model][k][t,i_mon,:]=coeff_0[model][i_mon,:]+coeff_1[model][i_mon,:]*emu_res[model][k][t-1,11,:]+innov_emu[i_mon][k,t]

            else:
                emu_res[model][k][t,i_mon,:]=coeff_0[model][i_mon,:]+coeff_1[model][i_mon,:]*emu_res[model][k][t,i_mon-1,:]+innov_emu[i_mon][k,t]

    emu_res[model][k]=emu_res[model][k][buffer:,:,:]

    for i_mon in range(12):

        emu_res[model][k][:,i_mon,:]=power_inv_transform(power_trans[model][i_mon],emu_res[model][k][:,i_mon,:],y_all[model])

joblib.dump(emu_res,dir_out_data_mod+'%i_emulator_innovations_fmin_log.pkl'%(nr_emus))
time_taken = dt.datetime.now() - start

print(‘time taken to create %i emulations: ’%(nr_emus*len(runs)), time_taken)
